In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import creep_event_picker as cep
import obspy
import matplotlib.dates as mdates
import cmcrameri.cm as cmc
import math
%matplotlib qt


In [ ]:
def check_dir(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path, exist_ok=True)

Import creepmeter database

In [ ]:
Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
Creepmeter_dataframe

In [ ]:
cutter = 2
n=10
cmap = cmc.roma #select the colormap

slicedCM = cmap(np.linspace(0, 1, cutter+1)) #list of colors that make up the colormap, can be called by using slicedCM[j] where j is the loop number in the fore loop

In [ ]:
frequency_dict = {
        1/60: '1S',    # 1 second
        1/6: '10S',  # 10 seconds
        1/2: '30S',  # 30 seconds
        1: '1T',  # 1 minute
        2: '2T',  # 2 minutes
        5: '5T',  # 5 minutes
        10: '10T',# 10 minutes
        15: '15T', # 15 minutes
        30: '30T',# 30 minutes
        60: '60T'    # 60 minutes
    }

In [7]:
plt.close('all')
for i in range(len(Creepmeter_dataframe)):
    file_code = Creepmeter_dataframe.File_code.iloc[i]
    path = "../../Data/DATA_tidied/SAC/{q}.SAC".format(q =file_code)
    path_save = "../../Data/DATA_tidied/Picks/"
    check_dir(path_save)
    print(path)
    try:
        df_picks = pd.read_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =file_code),index_col=0)
        column_names_list = df_picks.columns.tolist()
        if 'Quality_check_new' not in column_names_list:
            print(file_code,'no QC')
        
        
            st = obspy.read(path)
            tr = st[0]
            #tr.interpolate(sampling_rate=tr.stats.sampling_rate)
            tm = tr.times("utcdatetime")
            creep = tr.data
            sample_rate =  round(tr.stats.delta)/60
            tm_dt = []
            for w in range(len(tm)):
                tm_new = tm[w].datetime
                tm_dt.append(tm_new)
            print('now interpolate')
            '''tm_int = np.array(tm_dt)
            creep_int = creep'''
            tm_int, creep_int, upsampled = cep.interpolate(tm_dt,creep,sample_rate)

        
            df_picks.sort_values(by='ST',inplace=True)
            try:
                df_picks.reset_index(inplace=True,drop=True)
            except:
                dummy=10
            
        
            df_end=pd.DataFrame(columns=['ET'])
            df_start=pd.DataFrame(columns=['ST'])
            QC = []
            for p in range(math.ceil(len(df_picks)/cutter)):
                fig=plt.figure(figsize=(20,10))
                fig.tight_layout()
                ax = plt.subplot(1,1,1)

                df_selecta = df_picks.copy(deep=True)
                boolarr = np.logical_and(df_selecta.index >= cutter*(p),df_selecta.index < cutter*(p+1))
                df_selecta = df_selecta[boolarr]
                #print(df_selecta)
                fig.suptitle('events {a} - {b} of {c}'.format(a = df_selecta.index[0]+1,b= df_selecta.index[-1]+1,c =len(df_picks) ))
                boolarr_selecta = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_selecta.ST.iloc[0]).replace(tzinfo=None)\
                                                -dt.timedelta(days=30),\
                                                        np.array(tm_int)<=pd.to_datetime(df_selecta.ET.iloc[-1]).replace(tzinfo=None)\
                                                            +dt.timedelta(days = 120))
                tm_selecta = tm_int[boolarr_selecta]
                creep_selecta = creep_int[boolarr_selecta]
                plt.plot(tm_selecta,creep_selecta,marker='o', color='k', linestyle='-', linewidth=2, markersize=6)

                for j in range(len(df_picks)):
                    boolarr = np.logical_and(np.array(tm_selecta)>=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None),\
                                                    np.array(tm_selecta)<=pd.to_datetime(df_picks.ET.iloc[j]).replace(tzinfo=None))
                    event_creep = creep_selecta[boolarr]
                    event_time = tm_selecta[boolarr]
                    #print(event_time)
                    ax.plot(event_time,event_creep,color='orange',linewidth=3)
                    try:
                        ax.text(event_time[0],event_creep[0],'{k}/{n}'.format(k=df_picks.index[j]+1,n=len(df_picks)),color='green')
                        ax.text(event_time[-1],event_creep[-1],'{k}/{n}'.format(k=df_picks.index[j]+1,n=len(df_picks)),color='red')
                    except IndexError:
                        dummy=10

                ax.set_xlim([tm_selecta[0],tm_selecta[-1]])
                ax.set_ylim([np.nanmin(np.isfinite(creep_selecta) * creep_selecta), 
                    np.nanmax(np.isfinite(creep_selecta) * creep_selecta)])
                
                for j in range(len(df_selecta)):
                    if 'Quality_check' not in df_selecta.columns.to_list():
                        color='blue'
                    elif df_selecta["Quality_check"].iloc[j]=="CE":
                        color='green'
                    elif df_selecta["Quality_check"].iloc[j]=="NCE":
                        color="red"

                    boolarr = np.logical_and(np.array(tm_selecta)>=pd.to_datetime(df_selecta.ST.iloc[j]).replace(tzinfo=None),\
                                                    np.array(tm_selecta)<=pd.to_datetime(df_selecta.ET.iloc[j]).replace(tzinfo=None))
                    event_creep = creep_selecta[boolarr]
                    event_time = tm_selecta[boolarr]
                    #print(event_time)
                    ax.plot(event_time,event_creep,color=color,linewidth=j+1,zorder=100-(j*20))
                    ax.scatter([event_time[0]],[event_creep[0]],c='green',marker='s',s=20)
                    ax.scatter([event_time[-1]],[event_creep[-1]],c='red',zorder=10,s=10)
                    ax.text(event_time[0],event_creep[0],'{k}/{n}'.format(k=df_selecta.index[j]+1,n=len(df_picks)),color='green')
                    ax.text(event_time[-1],event_creep[-1],'{k}/{n}'.format(k=df_selecta.index[j]+1,n=len(df_picks)),color='red')

                
                ##### define start, end and QC ####
                
                
                scale = 1.1
                zp = cep.ZoomPan()
                figZoom = zp.zoom_factory(ax, base_scale = scale)
                figPan = zp.pan_factory(ax)
                s = 1
                fig.set_size_inches(11,11)
                fig.tight_layout()
                plt.show()
                for m in range(len(df_selecta)):
                    print('{m}/{n}'.format(m=df_selecta.index[m]+1,n=len(df_picks)))

                    print('is the start pick good?\n esc=yes, click=no')
                    check_start = plt.ginput(1,timeout = -1)
                    if len(check_start) ==1:
                        for sublist in check_start:
                            df = pd.DataFrame({'ST':[sublist[0]]})
                            df_start = pd.concat([df_start,df])
                        print('new start picked\n {k}'.format(k=mdates.num2date(sublist[0])))
                    else:
                        print('start all good! :D')
                        df = pd.DataFrame({'ST':mdates.date2num([df_selecta.ST.loc[df_selecta.index[m]]])})
                        df_start = pd.concat([df_start,df])

                    print("now pick the end\n esc=keep current, click=pick new")
                    pts = plt.ginput(1,timeout = -1)
                    if len(pts)==1:
                        for sublist in pts:
                            df = pd.DataFrame({'ET':[sublist[0]]})
                            df_end = pd.concat([df_end,df])
                            print('new end picked\n {k}'.format(k=mdates.num2date(sublist[0])))
                    else:
                        print('end all good! :D')
                        df = pd.DataFrame({'ET':mdates.date2num([df_selecta.ET.loc[df_selecta.index[m]]])})
                        df_end = pd.concat([df_end,df])
                    
                    
                    print('is this a creep event?\n esc=no, click=yes')
                    check_pts = plt.ginput(1,timeout = -1)
                    
                    if len(check_pts) == 1:
                        QC.append('CE')
                        print('YES!! :D')
                    else:
                        QC.append('NCE')
                        print('Nope :(')

                plt.close()
            rounder = frequency_dict[sample_rate]
            for r in range(len(df_end)):
                time_convert = mdates.num2date(float(df_end.ET.iloc[r]))
        
                s3 = pd.Series(time_convert).dt.round(rounder)
                df_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)  

            
            for r in range(len(df_start)):
                time_convert = mdates.num2date(float(df_start.ST.iloc[r]))
                s3 = pd.Series(time_convert).dt.round(rounder)
                df_start.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None) 
            
            #df_picks_end.ET= mdates.num2date(df_picks_end.ET)
            df_end.reset_index(inplace=True)
            df_start.reset_index(inplace=True)
            df_end.drop(columns=['index'],inplace=True)
            df_start.drop(columns=['index'],inplace=True)
            
            df_picks['ST_new'] = df_start['ST']
            df_picks['ET_new'] = df_end['ET']
            df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
            df_picks['ST_new'] = df_picks['ST_new'].astype(str).str[0:19]
            df_picks['ET_new'] = df_picks['ET_new'].astype(str).str[0:19]
            df_picks['Quality_check_new'] = QC
            df_picks.to_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =file_code))
        else:
            print(file_code,'QC done')
    except FileNotFoundError:
        print(file_code,'NO EVENTS')

../../Data/DATA_tidied/SAC/BAL1_1T.SAC
BAL1_1T QC done
../../Data/DATA_tidied/SAC/C461_10T.SAC
C461_10T no QC
now interpolate
1/63
is the start pick good?
 esc=yes, click=no
start all good! :D
now pick the end
 esc=keep current, click=pick new


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_14981/2936940463.py:120: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_start = pd.concat([df_start,df])


end all good! :D
is this a creep event?
 esc=no, click=yes


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_14981/2936940463.py:132: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_end = pd.concat([df_end,df])


YES!! :D
2/63
is the start pick good?
 esc=yes, click=no
new start picked
 2004-10-02 00:40:20.930533+00:00
now pick the end
 esc=keep current, click=pick new
new end picked
 2004-10-04 09:18:48.526394+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
3/63
is the start pick good?
 esc=yes, click=no
new start picked
 2004-10-04 09:20:01.819435+00:00
now pick the end
 esc=keep current, click=pick new
new end picked
 2004-10-06 15:59:53.810252+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
4/63
is the start pick good?
 esc=yes, click=no
new start picked
 2004-10-06 15:59:53.810252+00:00
now pick the end
 esc=keep current, click=pick new
new end picked
 2004-10-09 03:09:22.049911+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
5/63
is the start pick good?
 esc=yes, click=no
new start picked
 2004-10-09 03:09:38.611336+00:00
now pick the end
 esc=keep current, click=pick new
new end picked
 2004-10-13 04:50:27.100485+00:00
is this a creep event?
 esc=no, click=yes


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_14981/2936940463.py:151: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2004-10-02 00:40:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_14981/2936940463.py:157: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2004-09-29 13:40:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_start.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)


CTM1_10T QC done
../../Data/DATA_tidied/SAC/CTM1_60T.SAC
CTM1_60T QC done
../../Data/DATA_tidied/SAC/CWC3_10T.SAC
CWC3_10T QC done
../../Data/DATA_tidied/SAC/CWN1_1T.SAC
CWN1_1T QC done
../../Data/DATA_tidied/SAC/CWN1_10T.SAC
CWN1_10T QC done
../../Data/DATA_tidied/SAC/CWN1_2T.SAC
CWN1_2T NO EVENTS
../../Data/DATA_tidied/SAC/DS30_60T.SAC
DS30_60T QC done
../../Data/DATA_tidied/SAC/DU30_1T.SAC
DU30_1T QC done
../../Data/DATA_tidied/SAC/DU30_10T.SAC
DU30_10T QC done
../../Data/DATA_tidied/SAC/DU30_30T.SAC
DU30_30T NO EVENTS
../../Data/DATA_tidied/SAC/DU30_5T.SAC
DU30_5T QC done
../../Data/DATA_tidied/SAC/ESZ1_1T.SAC
ESZ1_1T QC done
../../Data/DATA_tidied/SAC/ESZ1_5T.SAC
ESZ1_5T QC done
../../Data/DATA_tidied/SAC/FCR1_1T.SAC
FCR1_1T QC done
../../Data/DATA_tidied/SAC/FE30_10T.SAC
FE30_10T QC done
../../Data/DATA_tidied/SAC/FE30_15T.SAC
FE30_15T QC done
../../Data/DATA_tidied/SAC/FE30_30T.SAC
FE30_30T QC done
../../Data/DATA_tidied/SAC/FE30_5T.SAC
FE30_5T NO EVENTS
../../Data/DATA_tidied/S

In [ ]:
18 et is 19 st